In [15]:
import data_processing.port_n_vessels as port
import data_processing.weather as weather
import data_processing.port_tracker as tracker
from data_processing.data_transform import DataTransformer

import numpy as np
import sqlite3
from datetime import datetime, timedelta
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup


In [16]:
from data_processing.data_transform import DataTransformer

In [17]:
hourly_weather_data = weather.openmeteo_hourly()
port_calls = tracker.port_calls()
port_berthed_f1 = tracker.inport() #Another reference

In [18]:
data_transformer = DataTransformer()

c:\Users\João Vitor\Desktop\Projetos\Santos_Port_n_Vessels\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.portodesantos.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\João Vitor\Desktop\Projetos\Santos_Port_n_Vessels\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.portodesantos.com.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\João Vitor\Desktop\Projetos\Santos_Port_n_Vessels\venv\lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.portodesantos.com.br'. Adding certificate verification is strongly advised. See: https

In [19]:
g_scheduled = data_transformer.get_scheduled()
g_foreseen_cargo = data_transformer.get_foreseen_cargo()
g_foreseen_cruise = data_transformer.get_foreseen_cruise()
g_berthage_ships = data_transformer.get_berthage_ships()
g_berthed_ships = data_transformer.get_berthed_ships()

In [34]:
conn = sqlite3.connect('data/raw_pvs.db')
cursor = conn.cursor()

In [21]:
cursor.execute('''

INSERT OR REPLACE INTO dim_ship (imo, mmsi, ship_name)

    SELECT imo, mmsi, ship_name FROM port_berthed_ships
    UNION
    SELECT imo, mmsi, ship_name FROM port_calls


''')

In [22]:
hourly_weather_data.to_sql('weather_data', conn, if_exists='append', index=False)

216

In [23]:
port_calls.to_sql('port_calls', conn, if_exists='append', index=False)

100

In [24]:
port_berthed_f1.to_sql('port_berthed_ships', conn, if_exists='append', index=False)

88

In [25]:
g_scheduled.to_sql('scheduled', conn, if_exists='append',index=False)
g_foreseen_cargo.to_sql('foreseen_cargo', conn, if_exists='append',index=False)
g_foreseen_cruise.to_sql('foreseen_cruise', conn, if_exists='append',index=False)
g_berthage_ships.to_sql('berthage_ships', conn, if_exists='append',index=False)
g_berthed_ships.to_sql('berthed_ships', conn, if_exists='append',index=False)

92

In [36]:
conn.commit()
conn.close()